# <font color='blue'>Files -- astropy.io.ascii</font>

<font color='blue'>The basic functionality of the `.read()` and `.write()` methods are sufficient for many purposes, but sometimes you have more complex files that you want to read in quickly, or large datasets you want to save with associated information like header keywords and metadata. `astropy.io.ascii` is a very powerful module for working with text files in python. While it uses the same basic methods discussed previously, they are hidden underneath convinent wrappers that we can use (and promptly forget how to open a text file on our own...)<br><br>

Let's start by using the `ascii.read()` function to read the contents of the VizieR table we were working with earlier.</font>

In [ ]:
from astropy.io import ascii
data = ascii.read('vizier-example.tsv')

<font color='blue'>Unfortunately not so simple! Depending on the formatting of your file, you may need to tell `read()` what the delimiter is, where the header and data start etc. In this case, it is confused by the delimiter, it thought the delimiter was a space, but it is infact a semi-colon.</font>

In [ ]:
data = ascii.read('vizier-example.tsv', delimiter=';')
data

<font color='blue'>`ascii.read` stores the data as an astropy `Table`.</font>

In [ ]:
print(type(data))

<font color='blue'>If the file has column headings, we can access the data in a specific column using the column names. This returns the column as an astropy `Column` object. The list of column headings can be accessed using the `.keys()` method

In [ ]:
print(data.keys())

<font color='blue'>If the table doesn't have a header or column headings, then they will be assigned default names (col0, col1...). Let's extract the first column of our table using the key:</font>

In [ ]:
hip_number = data['HIP']
print(type(hip_number))
print(hip_number)

<font color='blue'>If we wanted to use this column for some data analysis task, it is often worth converting it into a numpy array, either directly, or using the `.data` attribute of the `Column` object.</font>

In [ ]:
import numpy as np

print(type(np.array(data['HIP']))) # These two have the same result
print(type(data['HIP'].data))

print(np.array(data['HIP']))
print(data['HIP'].data)

<font color='blue'>Performing this conversion is especially important if you are using the data within long algorithms. It's possible to perform mathematical operations on a `Column` object, but that is much slower than on a numpy array. For example, if you are running a long MCMC chain and are performing an operation on a `Column` rather than an array, it could mean your code is slowed by a factor of several!</font>

<font color='blue'>Addressing individual rows of the table, or subsets of rows, is straightforward:</font>

In [ ]:
print(data[0])

In [ ]:
print(data[1])

In [ ]:
print(data[0:10])

<font color='blue'>We can also supply a list (tuple doesn't work) of indices that don't have to be sequential or unique:</font>

In [ ]:
index = [0, 10, 3, 5, 9, 9]
print(data[index])

<font color='blue'>We can also extract a specific column for this subset in two ways:</font>

In [ ]:
print(data[index]['HIP'].data)

In [ ]:
print(data['HIP'].data[index])

<font color='blue'>A full description of the `ascii.read()` options and functionality can be found here -- https://docs.astropy.org/en/stable/io/ascii/read.html. Probably the most useful ones are `format` to specify the type of table you are importing, `header_start` and `data_start` to indicate what line the header and data start on, respectively, `delimiter` as we used earlier, and `fill_values` to handle missing/bad data in the input table.</font>

***

<font color='blue'>The other main feature of the `ascii` module is the ability to write files from either arrays or lists using the `ascii.write()` function. Writing simple tables is quite straightfoward. Let's generate some data and write it to a file:</font>

In [ ]:
x = np.arange(10)
y = x ** 2
print([x, y])
ascii.write([x, y], 'my-new-table.txt')

<font color='blue'>Let's read it in again and confirm it is the same:</font>

In [ ]:
ascii.read('my-new-table.txt')

<font color='blue'>The data is all there, but notice how the column names are just the default (col0, col1, ...). When writing the table, we can specify the column names using the `names` keyword:</font>

In [ ]:
ascii.write([x, y], 'my-new-table.txt', names=['x', 'y'], overwrite=True)

In [ ]:
ascii.read('my-new-table.txt')

<font color='blue'>The two variables here have been saved as integers. Let's see what happens if `y` was a float instead:</font>

In [ ]:
x = np.arange(10)
y = x ** 0.5
print([x, y])
ascii.write([x, y], 'my-new-table.txt', names=['x', 'y'], overwrite=True)

In [ ]:
ascii.read('my-new-table.txt')

<font color='blue'>So `ascii.write()` detects the data type of the variable used for each column and saves it appropriately. We can also save things like strings:</font>

In [ ]:
x = np.arange(10)
y = x ** 0.5
z = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
print([x, y, z])
ascii.write([x, y, z], 'my-new-table.txt', names=['x', 'y', 'z'], overwrite=True)

In [ ]:
ascii.read('my-new-table.txt')

<font color='blue'>If we want to control how the output of a float is formatted, we can use the `formats` keyword. This accepts a dictionary of column name/format value pairs. This is most useful for specifying the number of decimal places you want for your floats.</font>

In [ ]:
x = np.arange(10)
y = x ** 0.5
z = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
print([x, y, z])
ascii.write([x, y, z], 'my-new-table.txt',
            names=['x', 'y', 'z'], overwrite=True,
            formats={'x': '{:d}', 'y': '{:.3f}', 'z': '{}'})

In [ ]:
with open('my-new-table.txt') as file:
    print(file.read())

<font color='blue'>Although this does mean you lose the precision when you want to read it in again...</font>

In [ ]:
ascii.read('my-new-table.txt')

<font color='blue'>The default format is usually sufficient for most needs, but others are available. `ecsv` is the one I use most often because it saves useful metadata (such as units, if you are using astropy's `units` module) in the header of the table:</font>

In [ ]:
x = np.arange(10)
y = x ** 0.5
z = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
ascii.write([x, y, z], 'my-new-table.txt', format='ecsv',
            names=['x', 'y', 'z'], overwrite=True,
            formats={'x': '{:d}', 'y': '{:.3f}', 'z': '{}'})

In [ ]:
with open('my-new-table.txt') as file:
    print(file.read())

<font color='blue'>and `latex` (I never want to manually write one of these ever again!):</font>

In [ ]:
x = np.arange(10)
y = x ** 0.5
z = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
ascii.write([x, y, z], 'my-new-table.txt', format='latex',
            names=['x', 'y', 'z'], overwrite=True,
            formats={'x': '{:d}', 'y': '{:.3f}', 'z': '{}'})

In [ ]:
with open('my-new-table.txt') as file:
    print(file.read())

## <font color='green'>Exercises</font>

<font color='green'>Create a new file that saves these three columns with an appropriate choice of data type/format. Optionally, give a name for each column so your output file has a header. After, check your file by reading it either with `open()` or `ascii.read()`.</font>

In [ ]:
fruit = ['apple', 'banana', 'orange', 'pear', 'strawberry']
weight = [70, 120, 90, 85, 15]
price = [0.9, 0.7, 0.4, 0.8, 0.05]



<font color='green'>When using the default output format the delimiter is a space. Change your code from above to use a different delimiter such as `,` or `;`. Also, use the `formats` keyword to control the formatting of the price so that it has two numbers after the decimal. Again, check the output by opening your file again.</font>

<font color='green'>Use `ascii.read()` to open `vizier-example2.tsv`. Remember that before we had to specifiy the delimiter. I had edited the previous VizieR file to remove a few extra rows between the column names and the data that contain the units and a horizontal line. See if you can use the `data_start` keyword to specify the line number (the line with the column names is line zero) so that the units and line are not included in the output.</font>

In [ ]:
ascii.read('vizier-example2.tsv', delimiter=';', data_start=3)

<font color='green'>Once you have done that, print the list of column names using the `.keys()` method</font>

<font color='green'>Using the column names and indexing, print the first ten entries in the `RA_ICRS` and `DE_ICRS` columns</font>